In [13]:
from os.path import join, exists, isfile, isdir, abspath, dirname, basename, realpath
from os import makedirs, listdir, pardir, getcwd

from dataclasses import dataclass, field
from typing import Union
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import logging as log
from enum import Enum

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Dropdown, Text, GridBox, GridspecLayout, VBox, HBox, Accordion, BoundedIntText, Checkbox, Layout, IntProgress, Tab, Image, Button
from IPython.display import display, Javascript

from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### Helper classes and functions

In [5]:
#--------------------------------------------------
# Helper classes and enums
#--------------------------------------------------
@dataclass(frozen=False, order=False)
class CentralityAlgorithm:
    """
    A data-class to store centrality measures arguments and other options
    """
    name: str = field(default="")
    func: object = field(default=print)
    args: dict = field(default_factory=dict)
    options: dict = field(default_factory=dict)

@dataclass(frozen=False, order=False)
class CentralityAlgorithmOutput:
    """
    A data-class to store centrality measures results
    """
    name: str = field(default="")
    result: dict = field(default_factory=dict)
    options: dict = field(default_factory=dict)

class GRAPH_TYPE(str, Enum):
    BOTH = "both"
    DIRECTED = "directed"
    def __str__(self) -> str:
        return self.value

#--------------------------------------------------
# Helper functions
#--------------------------------------------------
def convert_to_normal(snake_case: str) -> str:
    """
    Converts a snake case string to a normal title string

    Input:  "in_degree_centrality"
    Output: "In Degree Centrality"
    """
    components = snake_case.split('_')
    return " ".join(x.title() for x in components)

def print_log(text: str):
    """
    Prints the log
    """
    print(f"[ log ]: {text}")
    # log.info(f"[ log ]: {text}")

def print_error(text: str):
    """
    Prints the error
    """
    print(f"[ error ]: {text}")
    # log.warn(f"[ error ]: {text}")

def undirected_to_directed(graph: nx.Graph) -> nx.DiGraph:
    """
    Converts an undirected graph to a directed graph
    """
    di_graph = nx.DiGraph()
    di_graph.add_edges_from(graph.edges())
    return di_graph

def get_graph(graph: nx.Graph, graph_type: str) -> Union[nx.Graph, nx.DiGraph]:
    """
    Returns the graph based on the graph type
    """
    if graph_type == str(GRAPH_TYPE.DIRECTED):
        return undirected_to_directed(graph)
    elif graph_type == str(GRAPH_TYPE.BOTH):
        return graph
    else:
        print_error(f"Unknown graph type: {graph_type}")
        return graph

def downsample_connected_graph(graph: nx.Graph, largest_cc: set, sample_size: int = 25, verbose: bool = False, plot: bool = False) -> nx.Graph:
    """
    Down-samples the graph to only contain "sample_size" number of nodes
    """
    # pick a random node from the largest connected component
    __node = np.random.choice(list(largest_cc))
    __total_nodes, __graph = 0, nx.Graph()
    
    for i, _ in enumerate(range(1, sample_size)):
        # get the ego graph of the node
        __ego_graph = nx.ego_graph(graph, __node, radius=i)
        __total_nodes = len(__ego_graph)
        if (__total_nodes >= sample_size) and not nx.is_empty(__graph): break
        if verbose: print_log(f"Node: '{__node}' yields '{__total_nodes}' ego-graph nodes after radius '{i}'")
        __graph = __ego_graph
    
    if plot: nx.draw(__graph, with_labels=True)
    return __graph

def make_graph_connected(graph: nx.Graph, sample_size: int = 25, sorted_cc: list = list(), verbose: bool = False, plot: bool = False) -> nx.Graph:
    """
    Connect the graph by adding more edges between the nodes of connected components
    """
    if sample_size <= 0: raise ValueError(f"Invalid sample size: {sample_size}")
    if len(sorted_cc) <= 1: raise ValueError(f"Invalid connected component list")
    
    largest_cc = sorted_cc[0]
    __first_node = list(largest_cc)[0]

    __graph = nx.subgraph(graph, largest_cc).copy()

    for i, cc in enumerate(sorted_cc[1:]):
        # return if adding another connected component will make the graph bigger than sample size
        if (len(__graph) + len(cc)) > sample_size:
            if verbose: print_log(f"Graph size: {len(__graph)} + CC({i + 1}): {len(cc)} > Sample Size: {sample_size}")
            break
        # add new edge to make the graph connected
        if verbose: print_log(f"Adding edge between '{__first_node}' and '{list(cc)[0]}'")
        __graph.add_edge(__first_node, list(cc)[0])

        # add the new connected component to the graph
        if verbose: print_log(f"Adding {len(cc)} nodes from CC({i + 1}) to the graph")
        __graph.add_edges_from(nx.subgraph(graph, cc).edges())

    if plot:
        fig, axes = plt.subplots(nrows=1, ncols=2)
        axes[0].set_title("Original Graph")
        nx.draw(graph, with_labels=True, ax=axes[0])
        axes[1].set_title("Final Graph")
        nx.draw(__graph, with_labels=True, ax=axes[1])
    return __graph

def sample_graph(graph: Union[nx.Graph, nx.DiGraph], sample_size: int = 25, verbose: bool = False, plot: bool = False) -> Union[nx.Graph, nx.DiGraph]:
    """
    Returns a sample graph of the given 'sample_size'
    """
    if sample_size <= 0: raise ValueError(f"Invalid sample size: {sample_size}")

    is_directed = nx.is_directed(graph) and isinstance(graph, nx.DiGraph)
    if is_directed: graph = graph.to_undirected()

    sorted_cc = sorted(nx.connected_components(graph), key=len, reverse=True)
    largest_cc = sorted_cc[0]

    __graph = nx.Graph()

    if sample_size > len(largest_cc):
        if verbose: print_error(f"Sample size {sample_size} is greater than the largest connected component size {len(largest_cc)}")
        if nx.is_connected(graph):
            if verbose: print_log(f"Graph is connected")
            __graph = graph.copy()
        else:
            # make graph bigger by connecting the connected components
            if verbose: print_log(f"Making graph connected")
            __graph = make_graph_connected(graph=graph, sample_size=sample_size, sorted_cc=sorted_cc, verbose=verbose, plot=plot)
    else:
        # make graph shorter by keeping 'sample_size' elements only and connecting them
        if verbose: print_log(f"Downsampling a larger graph")
        __graph = downsample_connected_graph(graph=graph, largest_cc=largest_cc, sample_size=sample_size, verbose=verbose, plot=plot)
    
    return undirected_to_directed(__graph) if is_directed else __graph

def compute_algorithms(graph: nx.Graph, algorithms: list, verbose: bool) -> list:
    """
    Computes the centrality measures for the given graph
    """
    centrality_measures = list()
    for i, algorithm in enumerate(algorithms):
        algorithm_name = convert_to_normal(algorithm.name)
        graph_direction = algorithm.options["direction"]

        if verbose:
            print_log(f"Running '{algorithm_name}' ({i+1}/{len(algorithms)})")
            print_log(f"Option: '{algorithm.options}'")

        # if graph is not directed or undirected, then skip -> invalid direction option
        if graph_direction != str(GRAPH_TYPE.BOTH) and graph_direction != str(GRAPH_TYPE.DIRECTED):
            if verbose: print_error(f"Invalid direction: '{graph_direction}'")
            continue

        __graph = get_graph(graph=graph, graph_type=graph_direction)

        try:
            __centrality = algorithm.func(__graph, **algorithm.args)
        except Exception as e:
            if verbose: print_error(f"Error while running '{algorithm_name}': {e}")
            continue

        # return the degree as the weight
        if isinstance(__centrality, list):
            __centrality = {node: __graph.degree(node) for node in __centrality}

        centrality_measures.append(CentralityAlgorithmOutput(algorithm.name, __centrality, algorithm.options))

        if verbose:
            print_log(f"Result: '{__centrality}'")
            print_log(f"="*50)
    return centrality_measures

##### Global variables

In [ ]:
# list of all the algorithms
algorithms_to_run = [
    CentralityAlgorithm("degree_centrality", nx.degree_centrality, {}, {"direction":"both"}), 
    CentralityAlgorithm("in_degree_centrality", nx.in_degree_centrality, {}, {"direction":"directed"}), 
    CentralityAlgorithm("out_degree_centrality", nx.out_degree_centrality, {}, {"direction":"directed"}), 
    CentralityAlgorithm("eigenvector_centrality", nx.eigenvector_centrality, {"max_iter":100,"tol":1.0e-6,"weight":"weight"}, {"direction":"both"}), 
    CentralityAlgorithm("katz_centrality", nx.katz_centrality, {"alpha":0.1,"beta":1.0,"max_iter":1000,"tol":1.0e-6,"normalized":True,"weight":"weight"}, {"direction":"both"}), 
    CentralityAlgorithm("pagerank", nx.pagerank, {"alpha":0.85,"max_iter":100,"tol":1.0e-6,"weight":"weight"}, {"direction":"both"}), 
    CentralityAlgorithm("voterank", nx.voterank, {"number_of_nodes":10}, {"direction":"both"}), 
    CentralityAlgorithm("closeness_centrality", nx.closeness_centrality, {"distance":"weight"}, {"direction":"both"}), 
    CentralityAlgorithm("harmonic_centrality", nx.harmonic_centrality, {"distance":"weight"}, {"direction":"both"}), 
    CentralityAlgorithm("betweenness_centrality", nx.betweenness_centrality, {"k":None,"seed":7,"normalized":True,"weight":"weight"}, {"direction":"both"})
]

# # total sample size
# sample_size = 25
# verbose = False
# plot = False

# config. dir for input graph
parent_dir = abspath(join(join(getcwd(), pardir), pardir))
data_dir = join(parent_dir, 'data')
data_file = join(data_dir, 'DM-HT.txt')
# data_file = join(data_dir, 'DM-HT_small.txt')

graph = nx.read_edgelist(data_file, nodetype=str, data=(("weight", float),))
graph = sample_graph(graph=graph, sample_size=sample_size, verbose=verbose, plot=plot)
di_graph = undirected_to_directed(graph)

print(f"{graph}")

##### Interactive widget component(s)

In [12]:
#--------------------------
# widget configuration(s)
#--------------------------
data_files = [x for x in listdir(data_dir) if isfile(join(data_dir, x))]
algos = [x.name for x in algorithms_to_run]
sample_size = 25
show_algo_comparsion=True
show_dataset_comparsion=False
show_verbose=False

#------------------------------------
# Interactive Widget component(s)
#------------------------------------
w_dataset = Dropdown(options=data_files, value=data_files[0], description='Dataset ', disabled=False)
w_algo_list = Dropdown(options=algos, value=algos[0], description='Algorihtm ', disabled=False)
w_sample_size = BoundedIntText(value=sample_size, description='Sample Size ', min=1, max=1000, step=1, disabled=False)

w_show_algo_comparsion = Checkbox(value=show_algo_comparsion, description='Show Algo. Comparsion', disabled=False)
w_show_dataset_comparsion = Checkbox(value=show_dataset_comparsion, description='Show Dataset Comparsion', disabled=False)
w_show_verbose = Checkbox(value=show_verbose, description='Show Verbose', disabled=False)


# def create_expanded_button(description, button_style):
#     return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

# grid = GridspecLayout(3, 2, height='auto', width='auto')

# grid[0, 0] = w_dataset
# grid[0, 1] = w_algo_list
# grid[1, 0] = w_sample_size
# grid[1, 1] = w_show_algo_comparsion
# grid[2, 0] = w_show_dataset_comparsion
# grid[2, 1] = w_show_verbose

# grid

meta_data = [ w_dataset, w_algo_list, w_show_algo_comparsion, w_show_dataset_comparsion, w_show_verbose ]

GridBox(meta_data, layout=Layout(grid_template_columns="repeat(3, 33.34%)"))

GridBox(children=(Dropdown(description='Dataset ', options=('DM-HT.txt', 'CE-HT.txt', 'DM-HT_small.txt'), valu…

#### Compute algorithms

In [6]:
centrality_measures = compute_algorithms(graph=di_graph, algorithms=algorithms_to_run, verbose=False)

[CentralityAlgorithmOutput(name='degree_centrality', result={'C26G2.1': 0.058823529411764705, 'F11C3.3': 0.1764705882352941, 'F23F1.8': 0.058823529411764705, 'T22D1.9': 0.23529411764705882, 'K12F2.1': 0.11764705882352941, 'F29G9.5': 0.5294117647058824, 'Y17G7B.10': 0.29411764705882354, 'ZK20.3': 0.058823529411764705, 'F56G4.5': 0.11764705882352941, 'C41C4.8': 0.1764705882352941, 'F43D9.4': 0.11764705882352941, 'H18N23.2': 0.11764705882352941, 'C06A1.1': 0.1764705882352941, 'F58G4.1': 0.11764705882352941, 'R06C7.10': 0.11764705882352941, 'T18D3.4': 0.11764705882352941, 'T20F5.2': 0.058823529411764705, 'Y71H2AM.5': 0.058823529411764705}, options={'direction': 'both'}),
 CentralityAlgorithmOutput(name='in_degree_centrality', result={'C26G2.1': 0.0, 'F11C3.3': 0.1764705882352941, 'F23F1.8': 0.0, 'T22D1.9': 0.11764705882352941, 'T20F5.2': 0.058823529411764705, 'Y71H2AM.5': 0.058823529411764705, 'K12F2.1': 0.0, 'F29G9.5': 0.11764705882352941, 'Y17G7B.10': 0.1764705882352941, 'C06A1.1': 0.058